In [35]:
#!pip install -U twitter-client-py
import streamlit as st

In [23]:
from bs4 import BeautifulSoup
import pandas as pd
import time
from dotenv import load_dotenv
import os
import requests
import pprint as ppp
from twitter_client_py import TwitterClient, AsyncTwitterClient

In [36]:
# Q de consultas que vamos (limite mensual = 1.000)
api_key = os.getenv('2e2c904e0cmshdbad92d97808688p1e798ajsna8b04a4dd68a')
client = AsyncTwitterClient(api_key=api_key, timeout=20, verbose=True)
rate_limit = client.rate_limit
print(f"Limit: {rate_limit.limit}, Remaining requests: {rate_limit.remaining}, Reset time: {rate_limit.reset}")

Limit: 0, Remaining requests: 0, Reset time: 0


La proxima celda corresponde al codigo para trabajar en una API con cantidad de consultas limitadas. Se me ocurre que lo que habria que hacer, para evitar bloqueos (o exceder las cuotas de consulta), seria tener varios user con varias apis, e ir tomando de a uno por vez.

In [17]:
url_tweets_search_api_01 = "https://twitter-x.p.rapidapi.com/search/"

user_search_phrase = 'Nike'

querystring = {"query":user_search_phrase,"section":'latest',"limit":"20", 'lang':'en'} 			# "section" predeterminado sin posibilidad de cambiar por el cliente (para traer el sentimiento en tiempo real)
# el parametro "lang" no funciona, hay que resolverlo en el EDA (PROBAR CON LA API XCOM)

headers = {
	"x-rapidapi-key": "2e2c904e0cmshdbad92d97808688p1e798ajsna8b04a4dd68a",
	"x-rapidapi-host": "twitter-x.p.rapidapi.com"
}

response_api_01 = requests.get(url_tweets_search_api_01, headers=headers, params=querystring)
entries_api_01 = response_api_01.json()['data']['search_by_raw_query']['search_timeline']['timeline']['instructions'][0]['entries']

def clean_entries_with_dates(list_of_elem):
    clean_data = []
    for element in list_of_elem:
        content = element.get('content',[])                                                                             # para extraer el contenido de 'content'
        item_content = content.get('itemContent',{})                                                                    # para extraer el contenido de 'itemContent'
        if 'tweet_results' not in item_content or 'result' not in item_content['tweet_results'] : continue              # que siga de largo con este elemento si no tiene 'tweet_result' o 'result'
        result = item_content['tweet_results']['result']                                                                # para extraer el contenido de 'tweet_results'
        if 'legacy' not in result or 'full_text' not in result['legacy']: continue                                      # que siga de largo con este elemento si no tiene 'full_text' o 'legacy'
        full_text = result['legacy']['full_text']                                                                       # para extraer el contenido de 'full_text'
        post_date = result['legacy']['created_at']
        clean_data.append((post_date, full_text))
    return clean_data

aux_prueba = clean_entries_with_dates(entries_api_01)
aux_prueba = pd.DataFrame(aux_prueba)
aux_prueba																						# Resultado final

,0,1
0,Sun Sep 29 16:09:46 +0000 2024,@tennisfan2222 @Nike 💯
1,Sun Sep 29 16:09:42 +0000 2024,@aa9skillz https://t.co/1vlCcgSAQg
2,Sun Sep 29 16:09:41 +0000 2024,★\n肌ラボ 白潤プレミアム 薬用浸透美白乳液 140mL +極潤サシェット付 【医薬部外品...
3,Sun Sep 29 16:09:36 +0000 2024,Check out Nike Mouse's video! #TikTok https://...
4,Sun Sep 29 16:09:33 +0000 2024,@GadgetMarchank おめがじぇ〜〜
5,Sun Sep 29 16:09:23 +0000 2024,Nike motto on my mind lately
6,Sun Sep 29 16:09:20 +0000 2024,@BO6CODInformer W
7,Sun Sep 29 16:08:58 +0000 2024,"@Sudharsan_AK10 Well, @Nike took down their ‘B..."
8,Sun Sep 29 16:08:50 +0000 2024,@RSpleeban @Nike Thank you the materials feel ...
9,Sun Sep 29 16:08:45 +0000 2024,@iamdakejr https://t.co/od91fnp7qF


In [37]:
response_api_01.json()

{'data': {'search_by_raw_query': {'search_timeline': {'timeline': {'instructions': [{'type': 'TimelineAddEntries',
       'entries': [{'entryId': 'tweet-1840431846980739283',
         'sortIndex': '1840431866016956416',
         'content': {'entryType': 'TimelineTimelineItem',
          '__typename': 'TimelineTimelineItem',
          'itemContent': {'itemType': 'TimelineTweet',
           '__typename': 'TimelineTweet',
           'tweet_results': {'result': {'__typename': 'Tweet',
             'rest_id': '1840431846980739283',
             'core': {'user_results': {'result': {'__typename': 'User',
                'id': 'VXNlcjoxMzY1MzcyNjA5OTAzNDE5Mzk1',
                'rest_id': '1365372609903419395',
                'affiliates_highlighted_label': {},
                'has_graduated_access': True,
                'is_blue_verified': True,
                'profile_image_shape': 'Square',
                'legacy': {'can_dm': False,
                 'can_media_tag': True,
              